In [15]:
import os
import io
import re

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [21]:
import neuralcoref
import spacy

In [22]:
print('spacy version:', spacy.__version__)
print('neuralcoref version:', neuralcoref.__version__)

spacy version: 2.1.3
neuralcoref version: 4.0.0


In [4]:
def read_file(path):
    try:
        with io.open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except UnicodeDecodeError:
        try:
            with io.open(path, 'r', encoding='latin-1') as f:
                return f.read()
        except UnicodeDecodeError:
            return "Could not decode"
    

In [29]:
folder = "success"

# files sorted by number
files_ordered = sorted(os.listdir(folder), key=lambda name:int(name.split('_')[0]))

# an example file
file = files_ordered[1]
# name file
path = os.path.join(folder, file)

if os.path.isfile(path):
    content = read_file(path)
    print(f"Content of book ... {file}\n")
    print(content)

Content of book ... 2_Trilby_GeorgeDuMaurier_cleared.txt

Part First

    "Mimi Pinson est une blonde,
       Une blonde que l'on connaît;
     Elle n'a qu'une robe au monde,
       Landérirette! et qu'un bonnet!"


It was a fine, sunny, showery day in April.

The big studio window was open at the top, and let in a pleasant breeze
from the northwest. Things were beginning to look shipshape at last. The
big piano, a semi-grand by Broadwood, had arrived from England by "the
Little Quickness" (la Petite Vitesse, as the goods trains are called
in France), and lay, freshly tuned, alongside the eastern wall; on the
wall opposite was a panoply of foils, masks, and boxing-gloves.

A trapeze, a knotted rope, and two parallel cords, supporting each a
ring, depended from a huge beam in the ceiling. The walls were of the
usual dull red, relieved by plaster casts of arms and legs and hands and
feet; and Dante's mask, and Michael Angelo's altorilievo of Leda and the
swan, and a centaur and Lapith fr

In [13]:
paragraphs = content.split('\n\n')
print(paragraphs)

['Part First', '    "Mimi Pinson est une blonde,\n       Une blonde que l\'on connaît;\n     Elle n\'a qu\'une robe au monde,\n       Landérirette! et qu\'un bonnet!"', '\nIt was a fine, sunny, showery day in April.', 'The big studio window was open at the top, and let in a pleasant breeze\nfrom the northwest. Things were beginning to look shipshape at last. The\nbig piano, a semi-grand by Broadwood, had arrived from England by "the\nLittle Quickness" (la Petite Vitesse, as the goods trains are called\nin France), and lay, freshly tuned, alongside the eastern wall; on the\nwall opposite was a panoply of foils, masks, and boxing-gloves.', "A trapeze, a knotted rope, and two parallel cords, supporting each a\nring, depended from a huge beam in the ceiling. The walls were of the\nusual dull red, relieved by plaster casts of arms and legs and hands and\nfeet; and Dante's mask, and Michael Angelo's altorilievo of Leda and the\nswan, and a centaur and Lapith from the Elgin marbles--on none o

In [18]:
paragraphs_new = []
contents = []
chapters = []
old_idx = 0

for paragraph in paragraphs:
    # print("\n Paragraph: ", paragraph)
    
    lines = paragraph.split('\n')
    # print(lines)
    if re.match('[IVXLCDM\d]+[\.]*', lines[0].upper().strip()):
        if len(contents) == 0:
            contents = [" ".join(line.lower().split()) for line in lines]
            # print("contents", contents)
            
            if len(contents) == 0:
                print("Not found content", id_file)
                exit()
        continue
           
    if paragraph.lower().strip() in contents:
        if old_idx == 0:
            old_idx = len(paragraphs_new)
        else:
            current_idx = len(paragraphs_new)
            chapter = "\n\n".join(paragraphs_new[old_idx:current_idx])
            chapters.append(chapter)
            old_idx = current_idx
        continue
    
    if not re.match('.*[\w]+.*', paragraph):
        continue
    
    paragraphs_new.append(paragraph)
 

    if old_idx != 0:
        chapter = "\n\n".join(paragraphs_new[old_idx:len(paragraphs_new)])
        chapters.append(chapter)
        
print(len(paragraphs), len(paragraphs_new),
      #len(chapters)
     )    

2407 1987


In [23]:
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

In [34]:
geral_entities = []

for i, paragraph in enumerate(paragraphs_new[0:15]):
    paragraph = paragraph.replace('\n', ' ')
    paragraph = " ".join(paragraph.split())
    # print('\n', paragraph)****
    
    doc = nlp(paragraph)
    # print(doc)
    
    doc_coref = doc._.coref_resolved
    doc_coref = nlp(doc_coref)
    # print(doc_coref)
    
    entities = []
    for ent in doc_coref.ents:
        if ent.label_ == "PERSON":
            entities.append(ent.text)
    # print(entities)
    
    if len(entities) > 0:
        len_words = len(paragraph.split())
        geral_entities.extend(entities)
        # print('\n', paragraph)
        print(i, len_words, entities)
    
print("\nGeral Entities:\n", set(geral_entities))

1 20 ['Mimi Pinson', 'Landérirette']
2 72 ['Broadwood']
3 74 ['Dante', "Michael Angelo's", 'Leda']
4 39 ['Rembrandt', 'Tintoret', 'Leonardo da Vinci', 'Botticelli', 'Mantegna']
5 129 ['Barye']
9 86 ['the Man of Blood', 'the Man of Blood', 'Bare-armed', 'the Man of Blood', 'the Man of Blood', 'the Man of Blood', 'the Man of Blood', 'the Man of Blood']
11 104 ['Sothern']
13 101 ['Sandy']

Geral Entities:
 {"Michael Angelo's", 'Dante', 'Sandy', 'the Man of Blood', 'Bare-armed', 'Barye', 'Broadwood', 'Mimi Pinson', 'Sothern', 'Mantegna', 'Leda', 'Landérirette', 'Rembrandt', 'Leonardo da Vinci', 'Botticelli', 'Tintoret'}
